In [21]:
import pickle
import sqlparse

from django.test.client import RequestFactory
from django.db import connection, reset_queries

from cast.models import Blog
from cast.cache import PagedPostData

In [22]:
def show_queries(queries):
    for query in queries:
        formatted_sql = sqlparse.format(query['sql'], reindent=True, keyword_case='upper')
        print(formatted_sql)
        

def blocker(*args):
    raise Exception("No database access allowed here.")

    
# blog_slug = "ephes_blog"
blog_slug = "das_claas_und_nora_blog"
blog = Blog.objects.get(slug=blog_slug)
request_factory = RequestFactory()

In [23]:
%%time
reset_queries()
post_queryset = blog.unfiltered_published_posts
post_data = PagedPostData.create_from_blog_index_request(
    request=request_factory.get("/"),
    blog=blog,
)
print(len(connection.queries))

22
CPU times: user 59.4 ms, sys: 9.55 ms, total: 69 ms
Wall time: 79.9 ms


In [24]:
show_queries(connection.queries)

SELECT "wagtailcore_site"."id",
       "wagtailcore_site"."hostname",
       "wagtailcore_site"."port",
       "wagtailcore_site"."site_name",
       "wagtailcore_site"."root_page_id",
       "wagtailcore_site"."is_default_site",
       CASE
           WHEN ("wagtailcore_site"."hostname" = 'testserver'
                 AND "wagtailcore_site"."port" = 80) THEN 0
           WHEN ("wagtailcore_site"."hostname" = 'testserver'
                 AND "wagtailcore_site"."is_default_site") THEN 1
           WHEN "wagtailcore_site"."is_default_site" THEN 2
           ELSE 3
       END AS "match",
       "wagtailcore_page"."id",
       "wagtailcore_page"."path",
       "wagtailcore_page"."depth",
       "wagtailcore_page"."numchild",
       "wagtailcore_page"."translation_key",
       "wagtailcore_page"."locale_id",
       "wagtailcore_page"."latest_revision_id",
       "wagtailcore_page"."live",
       "wagtailcore_page"."has_unpublished_changes",
       "wagtailcore_page"."first_published_at",
 

In [4]:
#dir(post_data)

In [5]:
post_data.post_by_id

{522: <Post: März 2024-03-11>,
 521: <Post: März 2024-03-04>,
 517: <Post: Februar 2024-02-26>,
 516: <Post: Februar 2024-02-19>,
 513: <Post: Februar 2024-02-12>}

In [ ]:
from django.core import serializers

In [7]:
#data = serializers.serialize("json", post_data.queryset)

In [8]:
# data

In [9]:
post_data.post_queryset

<PageQuerySet [<Post: März 2024-03-11>, <Post: März 2024-03-04>, <Post: Februar 2024-02-26>, <Post: Februar 2024-02-19>, <Post: Februar 2024-02-12>]>

In [10]:
does_not_work = ("paginate_context")
# renditions_for_posts is gigantic

class SerializablePostData:
    def __init__(self, post_data):
        post_data_attrs = (
            "audios", "site", "template_base_dir", "root_nav_links", "theme_form",
            "videos", "renditions_for_posts", "owner_username_by_id", "images",
            "has_audio_by_id", "audios_by_post_id", "post_by_id", "queryset_data",
            #"paginate_context",
        )
        for attr in post_data_attrs:
            setattr(self, attr, getattr(post_data, attr))

In [11]:
serializable_post_data = SerializablePostData(post_data)

In [12]:
# serializable_post_data = post_data

In [13]:
%%time
cache_path = "paged_post_data.pkl"
with open(cache_path, "wb") as f:
    pickle.dump(serializable_post_data, f)

with open(cache_path, "rb") as f:
    loaded = pickle.load(f)

CPU times: user 26.3 ms, sys: 4.5 ms, total: 30.8 ms
Wall time: 28 ms


In [14]:
%%time
request = request_factory.get(blog.get_url())
request.htmx = False
reset_queries()

# with connection.execute_wrapper(blocker):
response = blog.serve(request, post_data=post_data)

CPU times: user 461 µs, sys: 18 µs, total: 479 µs
Wall time: 482 µs


In [15]:
print(len(connection.queries))

0


In [16]:
show_queries(connection.queries)

In [17]:
post = post_data.post_queryset[1]

In [18]:
post.body.raw_data

[{'id': '421e12ac-1634-42ba-b754-f2af52a54286', 'type': 'overview', 'value': [{'id': '83ed9ade-5209-43be-9e69-7fc99d7a9407', 'type': 'paragraph', 'value': '<p data-block-key="f3er9">Eher ereignisarme Woche. Claas war Anfang der Woche bei Leander und Ende der Woche bei Levi zu Besuch. Am Sonntag sind wir ins <a href="https://www.baeder-duesseldorf.de/baeder/hallenbaeder/muenster-therme/">Schwimmbad</a> gegangen, obwohl wir (besonders Katharina) immer noch krank waren, hat aber trotzdem Spaß gemacht.</p>'}, {'id': 'e1e0206c-2b06-45f8-9204-85c049471feb', 'type': 'gallery', 'value': {'layout': 'default', 'gallery': [{'id': '7e7c8e61-0338-4216-ac64-c6d2e49f80ae', 'type': 'item', 'value': 2397}, {'id': '0de05f0f-95ff-4eb4-a05f-fd618468517a', 'type': 'item', 'value': 2396}, {'id': 'c6f2d4e7-ae9b-4102-bb3e-5e8315c36cee', 'type': 'item', 'value': 2395}, {'id': 'bf0a198f-40eb-4680-a0d1-81646e1d5893', 'type': 'item', 'value': 2394}]}}]}]